In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import datetime as dt
start_time = dt.datetime.now()

# Create the Timestamp object 
ts = pd.Timestamp(year = 2011,  month = 11, day = 21, hour = 10, second = 49, tz = 'US/Central')  
today = ts.today()
today

Timestamp('2020-04-08 10:43:15.851581')

In [2]:
def job_listings(town, start, df, dupes):
    new_url = 'https://www.indeed.com/jobs?l={},+MA&radius=0&sort=date&sr=directhire&start={}'.format(town,start)
    today = pd.to_datetime('today')
    dupes = dupes
    new_start = start
    page = requests.get(new_url)
    html = BeautifulSoup(page.text, 'html.parser')
    html.find('td', attrs={'id': 'resultsCol'})
    html.find('div',attrs={'class','jobsearch-SerpJobCard'})
    results = html.find_all('div',attrs={'class','jobsearch-SerpJobCard'})
    
    job_already = 0
    new_jobs = 0
    
    for item in results:
        title = str(item.find('a',attrs={'class','jobtitle'}).contents)[4:-2]
        job_id = item['id']
        if job_id in df.index:
            job_already += 1
            if job_id not in dupes:
                dupes.append(job_id)
        else:
            new_jobs +=1
            try:
                company = str(item.find('span',attrs={'class','company'}).a.contents)[4:-2]
                company_rating = float(item.find('span',attrs={'class','ratingsContent'}).contents[0].strip())
                company_link = item.find('span',attrs={'class','company'}).a['href']
            except:
                try:
                    company = str(item.find('span',attrs={'class','company'}).contents)[4:-2]
                except:
                    company = None
                company_rating = None
                company_link = None
            df.loc[job_id,['title','company','company_link','company_rating', 'town', 'orig_date']] = [title, company,company_link,company_rating, town, today]
    if job_already == len(results):
        new_start += job_already
        return new_start, df, dupes
    else:
        new_start = len(df)
        return new_start, df, dupes

In [3]:
def town_results(town):
    df = pd.DataFrame(columns=['title','company','company_link','company_rating','town','orig_date'])
    dupes = []
    url = "https://www.indeed.com/jobs?as_and=&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&sr=directhire&as_src=&salary=&radius=0&l={}%2C+MA&fromage=any&limit=10&sort=date&psf=advsrch&from=advancedsearch".format(town)
    page = requests.get(url)
    
    html = BeautifulSoup(page.text, 'html.parser')
    html.find('td', attrs={'id': 'resultsCol'})
    html.find('div',attrs={'class','jobsearch-SerpJobCard'})
    page_results = html.find_all('div',attrs={'class','jobsearch-SerpJobCard'})
    num = 0
    
    for item in page_results:
        title = str(item.find('a',attrs={'class','jobtitle'}).contents)[4:-2]
        job_id = item['id']
        try:
            company = str(item.find('span',attrs={'class','company'}).a.contents)[4:-2]
            company_rating = float(item.find('span',attrs={'class','ratingsContent'}).contents[0].strip())
            company_link = item.find('span',attrs={'class','company'}).a['href']
        except:
            try:
                company = str(item.find('span',attrs={'class','company'}).contents)[4:-2]
            except:
                company = None
            company_rating = None
            company_link = None
        df.loc[job_id,['title','company','company_link','company_rating','town','orig_date']] = [title, company,company_link,company_rating, town, today]
        num += 1
        
    num_results = str(html.find('div', attrs={'id': 'searchCountPages'}).contents)
    results = int(num_results[num_results.find('of ')+3:num_results.find(' jobs')].replace(',',''))
    
    f = widgets.IntProgress(
        value=len(df),
        min=0,
        max=results,
        description='{}:'.format(town),
        bar_style='info',
        orientation='horizontal'
    )

    int_prog = widgets.HTML(
        value="{} out of {}".format(len(df),results)
    )
    
    display(f)
    display(int_prog)
    
    start = len(df)
    while start < results:
        start, df, dupes = job_listings(town, start, df, dupes)
        f.value = len(df)
        int_prog.value = "{} out of {}".format(len(df),results)
    return df, dupes



In [4]:
framingham_listings, framingham_dupes = town_results('Framingham')
natick_listings, natick_dupes = town_results('Natick')

NameError: name 'today' is not defined

In [ ]:
wayland_listings, wayland_dupes = town_results('wayland')
marlborough_listings, marlborough_dupes = town_results('marlborough')

In [ ]:
framingham_listings.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/framingham/framingham.csv')
#framingham_dupes.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/framingham/framingham_dupes.csv')

natick_listings.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/natick/natick.csv')
#natick_dupes.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/natick/natick_dupes.csv')

wayland_listings.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/wayland/wayland.csv')
#wayland_dupes.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/wayland/wayland_dupes.csv')

marlborough_listings.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/marlborough/marlborough.csv')
#marlborough_dupes.to_csv('/Users/williamgetman/Dropbox/Python/housing_prices/listings/marlborough/marlborough_dupes.csv')